In [1]:
import re
import json
import requests
from bs4 import BeautifulSoup
# path to your working directory
PATH = ""

In [2]:
response = requests.get("https://www.hplovecraft.com/writings/fiction/", headers={'User-Agent': 'Kula-Szpiegula'})
response.ok

True

In [3]:
digest = BeautifulSoup(response.content, "html.parser")

In [4]:
# The response is the table of contents, now we get all the separate stories
text_links = []
for tag in digest.find_all("a"):
    href = tag.get("href")
    if href.endswith(".aspx") and "/" not in href and href not in ["chrono.aspx", "publish.aspx"]:
        text_links.append( "https://www.hplovecraft.com/writings/texts/fiction/"+href )

This will be the preliminary dictionary holding the title and contents of each story.<br>
The extraction required inspection of the actual format, and replacement of almost all html tags,<br>
with the exception of newlines, which were present both as \<br\> and \r\n.<br>
We also print all the title and author data.<br>

In [5]:
lovdict = []
for i, link in enumerate(text_links[:]):
    response = requests.get(link, headers={'User-Agent': 'Kula-Szpiegula'})
    if response.ok:
        digest = BeautifulSoup(response.content, "html.parser")
        title = re.sub("<[^>]*>","", digest.find_all("font",attrs={"size":"+2"})[0].decode().replace("<br>","\n").replace("<br/>","\n").replace("</br>","\n") )
        print(f"{i:4d};", repr(title) )
        x = digest.find_all("div", attrs={"align":"justify"})
        y1 = x[1] if len(x)==2 else x[0]
        y2 = y1.decode().replace('\r\n',' ').replace('<br>','\n').replace('<br/>','\n').replace('</br>','\n')
        y2 = re.sub("<[^>]*>","",y2)
        lovdict.append( {"title":title, "content":y2.strip()} )
    else:
        print("❌ Request error")

   0; 'The Alchemist\nBy H. P. Lovecraft\n'
   1; 'Ashes\nBy C. M. Eddy, Jr.\nwith H. P. Lovecraft\n\n'
   2; 'At the Mountains of Madness\nBy H. P. Lovecraft\n'
   3; 'Azathoth\nBy H. P. Lovecraft\n'
   4; 'The Battle that Ended the Century\n(MS. Found in a Time Machine)\nBy R. H. Barlow\nwith H. P. Lovecraft\n\n\n'
   5; 'The Beast in the Cave\nBy H. P. Lovecraft\n'
   6; 'Beyond the Wall of Sleep\nBy H. P. Lovecraft\n'
   7; 'The Book\nBy H. P. Lovecraft\n'
   8; 'The Call of Cthulhu\nBy H. P. Lovecraft\n'
   9; 'The Case of Charles Dexter Ward\nBy H. P. Lovecraft\n'
  10; 'The Cats of Ulthar\nBy H. P. Lovecraft\n'
  11; 'Celephaïs\nBy H. P. Lovecraft\n'
  12; 'The Challenge from Beyond\nBy C.L. Moore, A. Merritt, H.P. Lovecraft,\nRobert E. Howard, and Frank Belknap Long\n'
  13; 'Collapsing Cosmoses\nBy R. H. Barlow\nand H. P. Lovecraft\n\n'
  14; 'The Colour out of Space\nBy H. P. Lovecraft\n'
  15; 'Cool Air\nBy H. P. Lovecraft\n'
  16; 'The Crawling Chaos\nBy H. P. Lovecraft\nwi

In [6]:
# save the stories in the original formatting
with open(PATH+"Lovecraft_stories.json", "w") as file:
    json.dump(lovdict, file, indent=1)

This is the second, almost final, version of the dictionary, where the title is separated from the authors.<br>
Most stories have [title] by [author(s)] at the beginning, but there is one exception.<br>
For now we separate those fields and strip away spaces and newlines.

In [7]:
lovdict2 = []
for k, story in enumerate(lovdict):
    lovdict2.append({})
    pre = '✅' if 'by' in story['title'].lower() else '❌'
    spl = re.split('(:?[Bb]y)', story['title'])
    clean = spl[0].replace('\n',' ').strip()
    auths = ' '.join( [q.replace('\n','').strip() for q in spl[1:]] ) 
    print(pre, f"{k:3d}", [clean, auths] )
    lovdict2[-1]['title'] = clean
    lovdict2[-1]['authors'] = auths
    sub1 = re.sub(' *(\n)+ *', '\n', story['content']) # repeated? newlines with possible spaces
    sub2 = re.sub( '  +', ' ', sub1 ) # repeated spaces
    lovdict2[-1]['content'] = sub2

✅   0 ['The Alchemist', 'By H. P. Lovecraft']
✅   1 ['Ashes', 'By C. M. Eddy, Jr.with H. P. Lovecraft']
✅   2 ['At the Mountains of Madness', 'By H. P. Lovecraft']
✅   3 ['Azathoth', 'By H. P. Lovecraft']
✅   4 ['The Battle that Ended the Century (MS. Found in a Time Machine)', 'By R. H. Barlowwith H. P. Lovecraft']
✅   5 ['The Beast in the Cave', 'By H. P. Lovecraft']
✅   6 ['Beyond the Wall of Sleep', 'By H. P. Lovecraft']
✅   7 ['The Book', 'By H. P. Lovecraft']
✅   8 ['The Call of Cthulhu', 'By H. P. Lovecraft']
✅   9 ['The Case of Charles Dexter Ward', 'By H. P. Lovecraft']
✅  10 ['The Cats of Ulthar', 'By H. P. Lovecraft']
✅  11 ['Celephaïs', 'By H. P. Lovecraft']
✅  12 ['The Challenge from Beyond', 'By C.L. Moore, A. Merritt, H.P. Lovecraft,Robert E. Howard, and Frank Belknap Long']
✅  13 ['Collapsing Cosmoses', 'By R. H. Barlowand H. P. Lovecraft']
✅  14 ['The Colour out of Space', 'By H. P. Lovecraft']
✅  15 ['Cool Air', 'By H. P. Lovecraft']
✅  16 ['The Crawling Chaos', 'By H

And deal with the only exception by hand.

In [8]:
lovdict2[88]['title'] = 'The Thing in the Moonlight'
lovdict2[88]['authors'] = 'By H. P. Lovecraft'
lovdict2[88]

{'title': 'The Thing in the Moonlight',
 'authors': 'By H. P. Lovecraft',
 'content': '“The Thing in the Moonlight” is based on a letter that Lovecraft wrote to Donald Wandrei on 24 November 1927. The story surrounding Lovecraft’s description of his dream was written by J. Chapman Miske and published in the January 1941 issue of Bizarre.\nLetter to Donald Wandrei\n“The Thing in the Moonlight”\nMy dreams occasionally approach’d the phantastical in character, tho’ falling somewhat short of coherence. One scene is especially stamp’d upon my recollection—that of a dank, fœtid, reed-choak’d marsh under a grey autumn sky, with a rugged cliff of lichen-crusted stone rising to the north. Impell’d by some obscure quest, I ascended a rift or cleft in this beetling precipice, noting as I did so the black mouths of many fearsome burrows extending from both walls into the depths of the stony plateau.\nMorgan is not a literary man; in fact he cannot speak English with any degree of coherency. That i

In [9]:
# save the final dictionary
with open(PATH+"Lovecraft_dict.json", "w") as file:
    json.dump(lovdict2, file, indent=1)

### The following is chunk production

strictly speaking no longer extractions - included for completeness

In [10]:
from chunkers import char_chunks, char_chunks_it

In [11]:
with open(PATH+"Lovecraft_dict.json", "r") as file:
    lovdict2 = json.load(file)

In [ ]:
# mean word length, for choosing the word/char chunks
wyrds = [len(q) for lo in lovdict2 for q in lo['content'].split(' ') ]
sum(wyrds)/len(wyrds)

4.823881555395185

As this example shows, the last chunk is a "tail" that will be visibly shorter with the linear chunking process.

In [20]:
char_chunks(lovdict2[56]['content'], length=1000, overlap=90);

18567 20.304395604395605 [121.65, 163.47619047619048]
1000 → 'Somewhere, to what remote and fearsome region I know not, Denys Barry has gone. I was with him the last night he lived among men, and heard his screams when the thing came to him; but all the peasants and police in County Meath could never find him, or the others, though they searched long and far. And now I shudder when I hear the frogs piping in swamps, or see the moon in lonely places.\nI had known Denys Barry well in America, where he had grown rich, and had congratulated him when he bought back the old castle by the bog at sleepy Kilderry. It was from Kilderry that his father had come, and it was there that he wished to enjoy his wealth among ancestral scenes. Men of his blood had once ruled over Kilderry and built and dwelt in the castle, but those days were very remote, so that for generations the castle had been empty and decaying. After he went to Ireland Barry wrote me often, and told me how under his care the grey

In [ ]:
# mean word length, for choosing the word/char chunks
wyrds = [len(q) for lo in lovdict2 for q in lo['content'].split(' ') ]
sum(wyrds)/len(wyrds)

4.823881555395185

So, we use the iterative chunking, which slowly increases chunk size and overlap<br>
to bring the last chunk's length to at least 95% of the mean length of the other chunks.<br>
For given length and overlap there is a theoretical number of chunks that fit into the whole text,<br>
which _should_ be integer, but isn't (generically). So we aim at the nearest integer,<br>
either up (by increaceasing overlap) or down (by increasing legth).

In [13]:
chunk_dict_list = []
for k, story in enumerate(lovdict2):
    print(f"{k:3d} ", end='', flush=True)
    loc_chunks = char_chunks_it(story['content'] , length=1000, overlap=90, verbose=False)
    L = len(loc_chunks)
    for i, chunk in enumerate(loc_chunks):
        chunk_dict_list.append( { 'story_id': k, 'title': story['title'], 'authors': story['authors'], 'len': L, 'chunk_id': i, 'text': chunk} )

  0 0.9697 final length: 1010, overlap: 118
  1 0.9697 final length: 1010, overlap: 98
  2 0.9765 final length: 1060, overlap: 122
  3 0.9656 final length: 1000, overlap: 170
  4 0.9682 final length: 1050, overlap: 94
  5 0.9602 final length: 1030, overlap: 90
  6 0.9653 final length: 1040, overlap: 118
  7 0.9775 final length: 1000, overlap: 98
  8 0.9678 final length: 1137, overlap: 142
  9 0.9761 final length: 1010, overlap: 98
 10 0.9598 final length: 1000, overlap: 90
 11 0.9535 final length: 1030, overlap: 98
 12 0.9983 final length: 1050, overlap: 146
 13 0.9924 final length: 1000, overlap: 90
 14 0.9688 final length: 1020, overlap: 98
 15 0.9819 final length: 1020, overlap: 90
 16 0.9895 final length: 1000, overlap: 114
 17 0.9682 final length: 1000, overlap: 94
 18 0.9720 final length: 1000, overlap: 118
 19 0.9586 final length: 1000, overlap: 98
 20 1.0021 final length: 1010, overlap: 90
 21 0.9848 final length: 1020, overlap: 102
 22 0.9706 final length: 1010, overlap: 90
 2

In [14]:
with open(PATH+'Lovecraft_chunks.json','w') as file:
    json.dump(chunk_dict_list, file, indent=1)